In [2]:
import os
from PIL import Image
import shutil
import imagehash

# 遍历文件夹及其子目录下的所有图片
def traverse_images(folder):
    image_paths = []
    for root, dirs, files in os.walk(folder):
        for filename in files:
            if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif', '.webp')):
                image_paths.append(os.path.join(root, filename))
    return image_paths

# 获取图片的像素大小信息
def get_image_pixel_info(img_path):
    try:
        with Image.open(img_path) as img:
            width, height = img.size
            return (height, width)
    except Exception as e:
        print(f"获取图片像素信息时出错: {img_path} - {e}")
        return None

# 去除源文件夹中的重复图片
def remove_duplicate_images(source_folder):
    # 统计每个子文件夹的处理情况
    folder_stats = {}  # {folder_path: (processed, deleted)}

    # 遍历源文件夹及其子目录下的所有图片
    all_image_paths = traverse_images(source_folder)

    # 按文件夹分组图片
    folder_images = {}
    for img_path in all_image_paths:
        folder = os.path.dirname(img_path)
        if folder not in folder_images:
            folder_images[folder] = []
        folder_images[folder].append(img_path)

    # 按文件夹处理图片
    for folder, image_paths in folder_images.items():
        # 存储哈希值和对应的文件路径
        hash_dict = {}
        processed = 0
        deleted = 0

        # 按排序后的图片处理
        for img_path in image_paths:
            processed += 1
            try:
                # 打开图片并计算其哈希值
                with Image.open(img_path) as img:
                    img_hash = imagehash.phash(img)
                    # 检查是否是重复图片
                    if img_hash in hash_dict:
                        # 删除重复图片
                        os.remove(img_path)
                        deleted += 1
                    else:
                        # 保留第一个出现的图片
                        hash_dict[img_hash] = img_path
            except Exception as e:
                # 如果处理失败，保留文件但不算作有效图片
                print(f"处理图片时出错: {img_path} - {e}")

        # 记录统计信息
        folder_stats[folder] = (processed, deleted)

    # 打印最终的统计信息
    print("去重完成 - 总计:")
    print(f"  处理文件夹: {len(folder_stats)}")
    print(f"  总处理图片: {sum(p for p, d in folder_stats.values())}")
    print(f"  总删除图片: {sum(d for p, d in folder_stats.values())}")
    
    # 逐个文件夹统计
    print("\n每个文件夹统计:")
    for folder, (processed, deleted) in folder_stats.items():
        print(f"  {os.path.relpath(folder, source_folder)}: 处理 {processed} 张，删除 {deleted} 张")

# 按横竖像素和分类图片并处理
def categorize_images_by_pixel_sum(source_folder, target_folder):
    # 创建目标文件夹（如果不存在）
    if not os.path.exists(target_folder):
        os.makedirs(target_folder)
    
    # 创建原始图片备份文件夹
    original_folder = os.path.join(target_folder, "original")
    if not os.path.exists(original_folder):
        os.makedirs(original_folder)
    
    # 获取源文件夹中所有图片
    all_image_paths = traverse_images(source_folder)
    
    # 备份原始图片
    for img_path in all_image_paths:
        # 获取文件名和扩展名
        filename = os.path.basename(img_path)
        file_root, file_ext = os.path.splitext(filename)
        
        # 创建新文件名（避免同名文件冲突）
        new_filename = f"{file_root}{file_ext}"
        new_path = os.path.join(original_folder, new_filename)
        count = 1
        while os.path.exists(new_path):
            new_filename = f"{file_root}_{count}{file_ext}"
            new_path = os.path.join(original_folder, new_filename)
            count += 1
        
        # 复制文件到原始图片备份文件夹
        shutil.copy2(img_path, new_path)
    
    print(f"原始图片已备份到 '{original_folder}' 文件夹。")
    
    # 创建一个字典来存储每个像素和对应的图片列表
    pixel_sum_dict = {}
    
    for img_path in all_image_paths:
        pixel_info = get_image_pixel_info(img_path)
        if pixel_info:
            height, width = pixel_info
            pixel_sum = height + width
            
            # 找到合适的分类
            found_category = False
            for existing_sum in pixel_sum_dict.keys():
                if abs(existing_sum - pixel_sum) <= 30:
                    pixel_sum_dict[existing_sum].append(img_path)
                    found_category = True
                    break
            
            if not found_category:
                pixel_sum_dict[pixel_sum] = [img_path]
    
    # 处理每个像素和对应的图片列表
    for pixel_sum, img_paths in pixel_sum_dict.items():
        # 如果一类图片的个数超过10个，则创建独立文件夹
        if len(img_paths) > 10:
            category_folder = os.path.join(target_folder, f"sum_{pixel_sum}")
            if not os.path.exists(category_folder):
                os.makedirs(category_folder)
        else:
            category_folder = target_folder
        
        # 分类为宽图和长图
        wide_images = [img_path for img_path in img_paths if get_image_pixel_info(img_path)[1] > get_image_pixel_info(img_path)[0]]
        tall_images = [img_path for img_path in img_paths if get_image_pixel_info(img_path)[0] > get_image_pixel_info(img_path)[1]]

        # 按宽度排序（宽图）
        wide_images.sort(key=lambda x: get_image_pixel_info(x)[1])
        # 按高度排序（长图）
        tall_images.sort(key=lambda x: get_image_pixel_info(x)[0])

        # 将排序后的图片移动到对应的文件夹
        for img_path in wide_images + tall_images:
            filename = os.path.basename(img_path)
            new_path = os.path.join(category_folder, filename)
            shutil.move(img_path, new_path)
    
    print(f"图片已按横竖像素和分类到 '{target_folder}' 文件夹。")

# 按目录结构重命名图片
def rename_images_by_folder_structure(source_folder):
    # 用于记录每个目录下的图片计数
    folder_counts = {}

    # 遍历源文件夹及其子目录下的所有图片
    for root, dirs, files in os.walk(source_folder):
        for filename in files:
            if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif', '.webp')):
                img_path = os.path.join(root, filename)
                
                # 获取相对路径（用于构建新文件名）
                relative_path = os.path.relpath(root, source_folder)
                path_parts = relative_path.split(os.sep)
                
                # 构建新文件名（目录结构_序号.扩展名）
                new_filename_parts = []
                for part in path_parts:
                    # 只保留字母、数字和下划线
                    part = ''.join(c if c.isalnum() or c == '_' else '_' for c in part)
                    new_filename_parts.append(part)
                
                # 获取当前目录的计数
                folder_count = folder_counts.get(root, 1)
                
                # 构建新文件名
                file_root, file_ext = os.path.splitext(filename)
                new_filename = f"{'_'.join(new_filename_parts)}_{folder_count}{file_ext}"
                new_path = os.path.join(root, new_filename)
                
                # 更新目录计数
                folder_counts[root] = folder_count + 1
                
                # 重命名文件
                try:
                    os.rename(img_path, new_path)
                    print(f"已重命名: {img_path} -> {new_path}")
                except Exception as e:
                    print(f"重命名文件时出错: {img_path} -> {new_path} - {e}")

# 删除目标文件夹中的空目录
def remove_empty_folders(target_folder):
    # 遍历目标文件夹及其子目录
    for root, dirs, files in os.walk(target_folder, topdown=False):
        for folder in dirs:
            folder_path = os.path.join(root, folder)
            if not os.listdir(folder_path):  # 如果目录为空
                os.rmdir(folder_path)  # 删除空目录
                print(f"已删除空目录: {folder_path}")

# 主程序
if __name__ == "__main__":
    source_folder = "result"  # 源文件夹
    target_folder = os.path.join(source_folder, "classified")


    # 执行按横竖像素和分类图片
    categorize_images_by_pixel_sum(source_folder, target_folder)

    # 执行去重
    remove_duplicate_images(source_folder)
    
    # 执行按目录结构重命名图片
    rename_images_by_folder_structure(target_folder)
    
    # 删除目标文件夹中的空目录
    remove_empty_folders(target_folder)

原始图片已备份到 'result\classified\original' 文件夹。
图片已按横竖像素和分类到 'result\classified' 文件夹。
去重完成 - 总计:
  处理文件夹: 13
  总处理图片: 2876
  总删除图片: 1346

每个文件夹统计:
  classified: 处理 35 张，删除 4 张
  classified\original: 处理 1686 张，删除 922 张
  classified\sum_1892: 处理 25 张，删除 12 张
  classified\sum_1997: 处理 248 张，删除 115 张
  classified\sum_2088: 处理 11 张，删除 5 张
  classified\sum_2133: 处理 436 张，删除 195 张
  classified\sum_2170: 处理 11 张，删除 5 张
  classified\sum_2204: 处理 20 张，删除 10 张
  classified\sum_2244: 处理 77 张，删除 38 张
  classified\sum_2276: 处理 33 张，删除 17 张
  classified\sum_2318: 处理 25 张，删除 12 张
  classified\sum_2409: 处理 22 张，删除 11 张
  classified\sum_2560: 处理 247 张，删除 0 张
已重命名: result\classified\ai-generated-7953404_1280.png -> result\classified\__1.png
已重命名: result\classified\ai-generated-7953406_1280.png -> result\classified\__2.png
已重命名: result\classified\ai-generated-8361970_1280.png -> result\classified\__3.png
已重命名: result\classified\ai-generated-8609619_1280.png -> result\classified\__4.png
已重命名: result\classified\

In [1]:
from PIL import Image
import os

def crop_images_to_min_dimensions(folder_path, output_folder):
    # 获取文件夹中所有图片文件
    files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif'))]
    
    if not files:
        print("文件夹中没有找到图片文件！")
        return

    # 找到最小的宽度和高度
    min_width = float('inf')
    min_height = float('inf')
    for file in files:
        img_path = os.path.join(folder_path, file)
        with Image.open(img_path) as img:
            width, height = img.size
            if width < min_width:
                min_width = width
            if height < min_height:
                min_height = height

    print(f"最小宽度为：{min_width}")
    print(f"最小高度为：{min_height}")

    # 创建输出文件夹
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # 裁剪所有图片
    for file in files:
        img_path = os.path.join(folder_path, file)
        with Image.open(img_path) as img:
            width, height = img.size
            # 计算裁剪的边界
            left = (width - min_width) // 2
            top = (height - min_height) // 2
            right = left + min_width
            bottom = top + min_height

            # 裁剪图片
            cropped_img = img.crop((left, top, right, bottom))

            # 如果图片是 RGBA 模式，转换为 RGB 模式
            if cropped_img.mode == 'RGBA':
                cropped_img = cropped_img.convert('RGB')

            output_path = os.path.join(output_folder, file)
            cropped_img.save(output_path)

        print(f"已裁剪并保存：{file}")

    print("所有图片裁剪完成！")

# 使用示例
folder_path = "result/剪映媒体"  # 替换为你的图片文件夹路径
output_folder = "result/剪映媒体/output"

crop_images_to_min_dimensions(folder_path, output_folder)

最小宽度为：1280
最小高度为：1280
已裁剪并保存：sum_2560_1.jpg
已裁剪并保存：sum_2560_10.png
已裁剪并保存：sum_2560_100.jpg
已裁剪并保存：sum_2560_101.jpg
已裁剪并保存：sum_2560_102.jpg
已裁剪并保存：sum_2560_102.png
已裁剪并保存：sum_2560_103.jpg
已裁剪并保存：sum_2560_104.png
已裁剪并保存：sum_2560_105.jpg
已裁剪并保存：sum_2560_105.png
已裁剪并保存：sum_2560_106.jpg
已裁剪并保存：sum_2560_107.jpg
已裁剪并保存：sum_2560_108.jpg
已裁剪并保存：sum_2560_108.png
已裁剪并保存：sum_2560_109.jpg
已裁剪并保存：sum_2560_109.png
已裁剪并保存：sum_2560_110.jpg
已裁剪并保存：sum_2560_111.png
已裁剪并保存：sum_2560_112.jpg
已裁剪并保存：sum_2560_113.jpg
已裁剪并保存：sum_2560_114.png
已裁剪并保存：sum_2560_115.jpg
已裁剪并保存：sum_2560_116.jpg
已裁剪并保存：sum_2560_116.png
已裁剪并保存：sum_2560_117.jpg
已裁剪并保存：sum_2560_117.png
已裁剪并保存：sum_2560_118.jpg
已裁剪并保存：sum_2560_119.jpg
已裁剪并保存：sum_2560_12.png
已裁剪并保存：sum_2560_120.jpg
已裁剪并保存：sum_2560_121.jpg
已裁剪并保存：sum_2560_122.jpg
已裁剪并保存：sum_2560_123.jpg
已裁剪并保存：sum_2560_124.jpg
已裁剪并保存：sum_2560_124.png
已裁剪并保存：sum_2560_125.jpg
已裁剪并保存：sum_2560_126.jpg
已裁剪并保存：sum_2560_127.jpg
已裁剪并保存：sum_2560_127.png
已裁剪并保存：sum_2560_128.png
已裁剪并保存：sum_2560_129.jp

In [12]:
import cv2
import numpy as np
import os
from pathlib import Path
import random
from datetime import datetime

def calculate_average_color(image_path):
    """
    计算图片的平均颜色
    """
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"无法读取图片：{image_path}。请检查图片路径和文件格式。")
    average_color = np.mean(image, axis=(0, 1)).astype(int)
    return average_color

def calculate_color_similarity(color1, color2):
    """
    计算两种颜色的相似度（欧几里得距离）
    """
    return np.linalg.norm(color1 - color2)

def select_images_by_color_similarity(image_paths, num_images=12, image_usage_count=None):
    """
    根据颜色相似度选择图片
    """
    if len(image_paths) < num_images:
        raise ValueError(f"图片数量不足{num_images}张，请确保文件夹中有足够的图片。")

    # 计算每张图片的平均颜色
    image_colors = {path: calculate_average_color(path) for path in image_paths}

    # 选择第一张图片作为参考
    reference_color = image_colors[random.choice(list(image_colors.keys()))]

    # 按颜色相似度排序
    sorted_images = sorted(image_colors.items(), key=lambda item: calculate_color_similarity(item[1], reference_color))

    # 选择前num_images张图片，同时检查使用次数
    selected_image_paths = []
    for path, _ in sorted_images:
        if image_usage_count[path] < 3:
            selected_image_paths.append(path)
            image_usage_count[path] += 1
        if len(selected_image_paths) == num_images:
            break

    if len(selected_image_paths) < num_images:
        raise ValueError("无法选择足够的图片，请检查图片数量和使用次数限制。")

    return selected_image_paths

def create_video_from_images(image_paths, output_video_path, total_duration=8, fps=60):
    """
    从指定的图片路径列表生成视频
    """
    # 调整图片大小并检查通道数
    processed_images, target_size = resize_and_check_images(image_paths)

    # 计算每张图片的展示时间
    total_frames = total_duration * fps  # 总帧数
    frames_per_image = total_frames // len(image_paths)  # 每张图片展示的帧数

    # 创建视频写入器
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video_writer = cv2.VideoWriter(output_video_path, fourcc, fps, target_size)

    # 将每张图片写入视频
    for img in processed_images:
        for _ in range(frames_per_image):
            video_writer.write(img)

    video_writer.release()
    print(f"视频已成功创建: {output_video_path}")

def resize_and_check_images(image_paths):
    """
    调整图片大小并检查通道数
    使用第一张图片的大小作为目标大小
    """
    resized_images = []

    # 获取第一张图片的大小作为目标大小
    first_image_path = image_paths[0]
    first_image = cv2.imread(first_image_path)
    
    if first_image is None:
        raise ValueError(f"无法读取图片：{first_image_path}。请检查图片路径和文件格式。")

    target_size = (first_image.shape[1], first_image.shape[0])

    for path in image_paths:
        img = cv2.imread(path)
        
        if img is None:
            raise ValueError(f"无法读取图片：{path}。请检查图片路径和文件格式。")

        # 调整图片大小
        if img.shape[:2] != target_size[::-1]:
            img = cv2.resize(img, target_size)

        # 检查通道数，确保是3通道
        if len(img.shape) == 2:  # 灰度图
            img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
        elif img.shape[2] == 4:  # RGBA图像
            img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)

        resized_images.append(img)

    return resized_images, target_size

if __name__ == "__main__":
    # 设置图片目录和输出目录
    image_directory = "images"  # 替换为你的图片目录
    output_directory = "output_videos"  # 输出视频文件夹

    # 创建输出目录（如果不存在）
    Path(output_directory).mkdir(parents=True, exist_ok=True)

    # 获取文件夹中所有图片文件
    image_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.webp']
    image_paths = [str(file) for file in Path(image_directory).iterdir() if file.suffix.lower() in image_extensions]

    if len(image_paths) < 12:
        raise ValueError("图片数量不足12张，请确保文件夹中有足够的图片。")

    # 确保每张图片至少出现一次，至多出现三次
    max_occurrences = 3
    image_usage_count = {path: 0 for path in image_paths}

    # 循环运行20次
    for i in range(3):
        # 使用当前时间生成输出视频文件名
        current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
        output_video_path = Path(output_directory) / f"{current_time}_output_video_{i+1}.mp4"

        try:
            # 选择颜色相近的图片
            selected_image_paths = select_images_by_color_similarity(image_paths, num_images=12, image_usage_count=image_usage_count)

            # 创建视频
            create_video_from_images(selected_image_paths, str(output_video_path), total_duration=8, fps=60)

        except Exception as e:
            print(f"生成视频时发生错误: {e}")

    # 输出每张图片的使用次数
    print("每张图片的使用次数：")
    for path, count in image_usage_count.items():
        print(f"{path}: {count}次")

视频已成功创建: output_videos\20250720_172531_output_video_1.mp4
视频已成功创建: output_videos\20250720_172536_output_video_2.mp4
视频已成功创建: output_videos\20250720_172541_output_video_3.mp4
每张图片的使用次数：
images\sum_2560_10.png: 2次
images\sum_2560_115.jpg: 1次
images\sum_2560_116.jpg: 1次
images\sum_2560_120.jpg: 1次
images\sum_2560_125.jpg: 1次
images\sum_2560_128.png: 2次
images\sum_2560_129.png: 2次
images\sum_2560_130.jpg: 1次
images\sum_2560_14.jpg: 1次
images\sum_2560_16.png: 1次
images\sum_2560_172.png: 0次
images\sum_2560_175.jpg: 0次
images\sum_2560_175.png: 2次
images\sum_2560_202.jpg: 2次
images\sum_2560_207.jpg: 1次
images\sum_2560_218.jpg: 1次
images\sum_2560_22.png: 2次
images\sum_2560_221.jpg: 0次
images\sum_2560_226.jpg: 2次
images\sum_2560_227.jpg: 1次
images\sum_2560_235.jpg: 1次
images\sum_2560_236.png: 2次
images\sum_2560_36.png: 1次
images\sum_2560_40.jpg: 1次
images\sum_2560_41.png: 0次
images\sum_2560_53.png: 2次
images\sum_2560_54.jpg: 1次
images\sum_2560_85.png: 2次
images\sum_2560_89.jpg: 1次
images\sum_256

In [3]:
import cv2
import numpy as np
import os
from pathlib import Path
from datetime import datetime

def resize_and_check_images(image_paths, target_size=None):
    """
    调整图片大小并检查通道数
    如果target_size为None，则使用第一张图片的大小作为目标大小
    """
    resized_images = []
    
    # 获取第一张图片的大小作为目标大小
    if target_size is None:
        first_image = cv2.imread(image_paths[0])
        target_size = (first_image.shape[1], first_image.shape[0])
    
    for path in image_paths:
        img = cv2.imread(path)
        
        # 调整图片大小
        if img.shape[:2] != target_size[::-1]:
            img = cv2.resize(img, target_size)
        
        # 检查通道数，确保是3通道
        if len(img.shape) == 2:  # 灰度图
            img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
        elif img.shape[2] == 4:  # RGBA图像
            img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)
        
        resized_images.append(img)
    
    return resized_images, target_size

def create_video_from_images(image_paths, output_video_path, fps=60, transition_duration=1):
    """
    将一组图片转换为视频，并添加多样化的过渡效果
    
    参数：
    image_paths: 图片文件路径列表
    output_video_path: 输出视频文件路径
    fps: 视频帧率，默认30fps
    transition_duration: 过渡效果持续时间（秒），默认1秒
    """
    
    # 调整图片大小并检查通道数
    processed_images, target_size = resize_and_check_images(image_paths)
    width, height = target_size
    
    # 创建视频写入器
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video_writer = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))
    
    # 每张图片显示的总帧数 = 过渡帧数 + 停留帧数
    transition_frames = int(transition_duration * fps)
    display_frames = int(0.05 * transition_frames)  # 每张图片显示2秒过渡时间
    
    # 定义不同的过渡效果
    transition_styles = [
        "fade"         # 淡入淡出
        # "slide"        # 平移过渡
        # "zoom",          # 缩放过渡
        # "rotate",        # 旋转过渡
        # "checkerboard",  # 棋盘格过渡
        # "door",          # 门过渡
        # "horizontal_blinds",  # 水平百叶窗
        # "vertical_blinds",    # 垂直百叶窗
        # "dissolve",      # 溶解效果
        # "flip",          # 翻转效果
        # "roll",          # 滚动效果
        # "grid_wipe",     # 网格擦除
        # "chaos"          # 混沌效果
    ]
    
    # 处理每张图片
    for i in range(len(processed_images) - 1):
        current_image = processed_images[i]
        next_image = processed_images[i+1]
        
        # 随机选择过渡效果
        transition_style = np.random.choice(transition_styles)
        
        # 添加图片展示部分
        for _ in range(display_frames):
            video_writer.write(current_image)
        
        # 根据选择的过渡风格应用相应的过渡效果
        if transition_style == "fade":
            # 淡入淡出效果
            for j in range(transition_frames):
                alpha = j / transition_frames
                blended = cv2.addWeighted(current_image, 1 - alpha, next_image, alpha, 0)
                video_writer.write(blended)
        
        elif transition_style == "slide":
            # 平移过渡效果
            for j in range(transition_frames):
                # 从右向左滑动
                shift = int(width * (1 - j / transition_frames))
                transition_frame = np.zeros_like(current_image)
                
                # 复制当前图片到左侧
                transition_frame[:, :shift] = current_image[:, :shift]
                
                # 复制下一张图片到右侧
                next_part = next_image[:, :width - shift]
                transition_frame[:, shift:shift + next_part.shape[1]] = next_part
                
                video_writer.write(transition_frame)
        
        elif transition_style == "zoom":
            # 缩放过渡效果
            for j in range(transition_frames):
                scale = 1 + 0.3 * (1 - j / transition_frames)  # 从放大到正常大小
                zoomed_current = cv2.resize(current_image, None, fx=scale, fy=scale)
                zoomed_next = cv2.resize(next_image, None, fx=scale, fy=scale)
                
                # 裁剪图像到原始大小
                height_zoomed, width_zoomed, _ = zoomed_current.shape
                start_y = (height_zoomed - height) // 2
                start_x = (width_zoomed - width) // 2
                
                current_cropped = zoomed_current[start_y:start_y+height, start_x:start_x+width]
                next_cropped = zoomed_next[start_y:start_y+height, start_x:start_x+width]
                
                # 混合两张图片
                alpha = j / transition_frames
                blended = cv2.addWeighted(current_cropped, 1 - alpha, next_cropped, alpha, 0)
                video_writer.write(blended)
        
        elif transition_style == "rotate":
            # 旋转过渡效果
            for j in range(transition_frames):
                angle = 30 * (1 - j / transition_frames)  # 从30度旋转到0度
                scale = 1
                
                # 旋转当前图片
                M = cv2.getRotationMatrix2D((width//2, height//2), angle, scale)
                rotated_current = cv2.warpAffine(current_image, M, (width, height))
                
                # 旋转下一张图片（反方向）
                M = cv2.getRotationMatrix2D((width//2, height//2), -angle, scale)
                rotated_next = cv2.warpAffine(next_image, M, (width, height))
                
                # 混合两张图片
                alpha = j / transition_frames
                blended = cv2.addWeighted(rotated_current, 1 - alpha, rotated_next, alpha, 0)
                video_writer.write(blended)
        
        elif transition_style == "checkerboard":
            # 棋盘格过渡效果
            for j in range(transition_frames):
                # 创建棋盘格掩码
                board_size = 60  # 棋盘格大小
                mask = np.zeros((height, width), dtype=np.uint8)
                
                # 填充棋盘格
                for y in range(0, height, board_size):
                    for x in range(0, width, board_size):
                        if (y // board_size + x // board_size) % 2 == 0:
                            if j / transition_frames > np.random.rand():
                                mask[y:y+board_size, x:x+board_size] = 255
                
                # 应用掩码
                transition_frame = np.where(mask[:, :, np.newaxis] == 255, next_image, current_image)
                video_writer.write(transition_frame)
        
        elif transition_style == "door":
            # 门过渡效果
            for j in range(transition_frames):
                # 创建门状掩码
                door_width = int(width * 0.5)  # 门的宽度
                shift = int(door_width * (j / transition_frames))
                
                mask = np.ones((height, width), dtype=np.uint8) * 255
                
                # 左门
                mask[:, :shift] = 0
                # 右门
                mask[:, width - shift:] = 0
                
                # 应用掩码
                transition_frame = np.where(mask[:, :, np.newaxis] == 255, next_image, current_image)
                video_writer.write(transition_frame)
        
        elif transition_style == "horizontal_blinds":
            # 水平百叶窗效果
            for j in range(transition_frames):
                blind_height = 20  # 百叶窗高度
                transition_frame = np.copy(current_image)
                
                for y in range(0, height, blind_height):
                    if j / transition_frames > np.random.rand():
                        # 替换为下一张图片的对应区域
                        transition_frame[y:y+blind_height, :] = next_image[y:y+blind_height, :]
                
                video_writer.write(transition_frame)
        
        elif transition_style == "vertical_blinds":
            # 垂直百叶窗效果
            for j in range(transition_frames):
                blind_width = 20  # 百叶窗宽度
                transition_frame = np.copy(current_image)
                
                for x in range(0, width, blind_width):
                    if j / transition_frames > np.random.rand():
                        # 替换为下一张图片的对应区域
                        transition_frame[:, x:x+blind_width] = next_image[:, x:x+blind_width]
                
                video_writer.write(transition_frame)
        
        elif transition_style == "dissolve":
            # 溶解效果
            for j in range(transition_frames):
                # 创建随机溶解掩码
                mask = np.random.rand(height, width) < (j / transition_frames)
                mask = mask.astype(np.uint8) * 255
                
                # 应用掩码
                transition_frame = np.where(mask[:, :, np.newaxis] == 255, next_image, current_image)
                video_writer.write(transition_frame)
        
        elif transition_style == "flip":
            # 翻转效果
            for j in range(transition_frames):
                # 水平翻转当前图片
                flip_ratio = j / transition_frames
                flipped = cv2.flip(current_image, 1)
                
                # 创建过渡帧
                transition_frame = cv2.addWeighted(current_image, 1 - flip_ratio, flipped, flip_ratio, 0)
                video_writer.write(transition_frame)
        
        elif transition_style == "roll":
            # 滚动效果
            for j in range(transition_frames):
                # 滚动图片
                roll_amount = int(height * (j / transition_frames))
                transition_frame = np.roll(current_image, roll_amount, axis=0)
                
                # 确保滚动后的图片形状与原始图片一致
                if transition_frame.shape == current_image.shape:
                    # 混合两张图片
                    alpha = j / transition_frames
                    blended = cv2.addWeighted(transition_frame, 1 - alpha, next_image, alpha, 0)
                    video_writer.write(blended)
        
        elif transition_style == "grid_wipe":
            # 网格擦除效果
            for j in range(transition_frames):
                # 创建网格擦除掩码
                grid_size = 20
                mask = np.zeros((height, width), dtype=np.uint8)
                
                for y in range(0, height, grid_size):
                    for x in range(0, width, grid_size):
                        if j / transition_frames > np.random.rand():
                            mask[y:y+grid_size, x:x+grid_size] = 255
                
                # 应用掩码
                transition_frame = np.where(mask[:, :, np.newaxis] == 255, next_image, current_image)
                video_writer.write(transition_frame)
        
        elif transition_style == "chaos":
            # 混沌效果
            for j in range(transition_frames):
                # 创建随机混沌掩码
                mask = np.random.rand(height, width) < (j / transition_frames)
                mask = mask.astype(np.uint8) * 255
                
                # 应用掩码
                transition_frame = np.where(mask[:, :, np.newaxis] == 255, next_image, current_image)
                video_writer.write(transition_frame)
    
    # 处理最后一张图片
    last_image = processed_images[-1]
    for _ in range(display_frames):
        video_writer.write(last_image)
    
    video_writer.release()
    print(f"视频已成功创建: {output_video_path}")

# 示例使用
if __name__ == "__main__":
    # 设置图片目录和输出路径
    image_directory = "images"  # 替换为你的图片目录

    # 使用当前时间生成输出视频文件名
    current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_video_path = f"{current_time}_output_video.mp4"

    # 获取目录下所有文件路径
    all_files = list(Path(image_directory).iterdir())

    # 过滤出常见图片格式的文件
    image_extensions = ['.jpg', '.jpeg', '.png', '.gif', '.bmp', '.webp']
    image_files = [file for file in all_files if file.suffix.lower() in image_extensions]

    # 对图片文件进行重命名
    for index, image_file in enumerate(image_files):
        # 获取文件的扩展名
        file_extension = image_file.suffix
        # 构造新的文件路径
        new_path = Path(image_directory) / f"{index}{file_extension}"
        # 重命名文件
        image_file.rename(new_path)
        print(f"Renamed: {image_file.name} -> {index}{file_extension}")

    # 获取所有图片文件路径并排序
    image_paths = []
    for ext in image_extensions:
        image_paths.extend(Path(image_directory).glob('*' + ext))
    image_paths = list(map(str, image_paths))
    image_paths.sort()  # 按名称排序

    print("Sorted image paths:")
    for path in image_paths:
        print(path)
    
    if not image_paths:
        print("图片目录中没有找到图片文件！")
    else:
        # 创建视频
        create_video_from_images(
            image_paths, 
            output_video_path, 
            fps=60, 
            transition_duration=1
        )

Renamed: 0.png -> 0.png
Renamed: 1.png -> 1.png
Renamed: 2.png -> 2.png
Renamed: 3.png -> 3.png
Sorted image paths:
images\0.png
images\1.png
images\2.png
images\3.png
视频已成功创建: 20250711_154128_output_video.mp4
